In [ ]:
import os
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector, AdaptiveDetector
# The new, correct way for PySceneDetect v0.6+
from scenedetect.video_splitter import split_video_ffmpeg


def find_and_split_scenes(video_path, output_dir):
    """
    Finds content-based scenes in a video and splits the video into
    separate files for each scene.

    Args:
        video_path (str): Path to the input video.
        output_dir (str): Folder to save the scene clips.
    """
    # 1. Open the video and create a SceneManager
    video = open_video(video_path)
    scene_manager = SceneManager()

    # 2. Add a detector. AdaptiveDetector is great for high-motion content.
    # The adaptive_threshold is the sensitivity of the detector relative to the
    # video's motion. A good starting point is 3.0. Lower values detect
    # more scenes, higher values detect fewer.
    scene_manager.add_detector(AdaptiveDetector(adaptive_threshold=3.0))

    # 3. Run the scene detection
    print(f"Detecting scenes in {os.path.basename(video_path)}...")
    scene_manager.detect_scenes(video=video, show_progress=True)

    # 4. Get the list of scenes found
    # Each scene is a tuple of (start_time, end_time)
    scene_list = scene_manager.get_scene_list()

    print(f"Found {len(scene_list)} scenes.")

    if not scene_list:
        print("No scenes detected.")
        return

    # 5. (Optional but Recommended) Split the video into clips
    # This will create a separate video file for each detected scene.
    os.makedirs(output_dir, exist_ok=True)
    print(f"Splitting video into scenes in folder: {output_dir}")
    
    # split_video_ffmpeg uses the command-line tool ffmpeg.
    # Ensure ffmpeg is installed and accessible in your system's PATH.
    try:
        split_video_ffmpeg(video_path, scene_list, output_dir=output_dir,
                           show_progress=True)
        print("Video successfully split into scenes.")
    except Exception as e:
        print(f"Error splitting video: {e}")
        print("Please ensure ffmpeg is installed and in your system's PATH.")



movie_path = r"movie\EmpuraanLucif2_TSR_TA-XX_4K-51-DOM_20250125_video-002.mxf"
output_folder = r"movie_scenes\EmpuraanLucif2_TSR_TA-XX_4K-51-DOM_20250125_video-002"

find_and_split_scenes(movie_path, output_folder)

c:\Users\DELL\anaconda3\envs\qube-cinemas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detecting scenes in 96.mkv...


  Detected: 2122 | Progress: 100%|█████████▉| 227672/227673 [22:09<00:00, 171.30frames/s]


Found 2123 scenes.
Splitting video into scenes in folder: movie_scenes\96-AdaptiveDetector


100%|██████████| 227672/227672 [42:46<00:00, 88.72frame/s] 

Video successfully split into scenes.


In [ ]:
import cv2
import os
import torch
import pyiqa
import numpy as np
from collections import deque
from tqdm import tqdm

# ==============================================================================
# Core Function to Find and Extract High-Quality Frame Sequence
# ==============================================================================

def find_first_sequence_musiq_niqe(
    video_path,
    output_folder,
    musiq_metric,
    niqe_metric,
    device,
    sequence_length=15,
    musiq_threshold=35.0,
    niqe_threshold=6.0
):
    """
    Finds the first sequence of high-quality frames in a video using a
    combination of MUSIQ and NIQE metrics, then saves the sequence and stops.
    """
    # --- 1. Initialization ---
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    frame_buffer = deque(maxlen=sequence_length)
    frame_count = 0
    base_video_name = os.path.basename(video_path).split('.')[0]
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"\n--- Processing {base_video_name} ({total_frames} frames) | MUSIQ > {musiq_threshold}, NIQE < {niqe_threshold} ---")

    # --- 2. Frame-by-Frame Processing Loop ---
    with tqdm(total=total_frames, desc=f"Scanning {base_video_name}", unit="frame") as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            pbar.update(1)

            # --- 2a. Pre-processing and Quality Checks ---
            # Skip blank/black frames to avoid corrupting the sequence
            if np.std(frame) < 10.0:
                frame_buffer.clear()
                continue

            # Convert frame to a tensor for the IQA models
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = torch.tensor(frame_rgb).permute(2, 0, 1).unsqueeze(0) / 255.0
            frame_tensor = frame_tensor.to(device)

            # Calculate quality scores
            with torch.no_grad():
                musiq_score = musiq_metric(frame_tensor).item()
                niqe_score = niqe_metric(frame_tensor).item()

            # --- 2b. Buffer Management ---
            # A frame is "good" if it passes both quality thresholds
            is_good_quality = (musiq_score >= musiq_threshold) and (niqe_score <= niqe_threshold)

            if is_good_quality:
                # Add good frames to the buffer
                frame_buffer.append({'frame_num': frame_count, 'frame': frame})
            else:
                # If a frame is of poor quality, the sequence is broken. Reset the buffer.
                frame_buffer.clear()

            # --- 2c. Save Sequence and Exit Condition ---
            # If the buffer is full, we have found a complete, high-quality sequence
            if len(frame_buffer) == sequence_length:
                print(f"\nSUCCESS: Found a valid sequence of {sequence_length} frames in {base_video_name}. Saving...")
                sequence_dir = os.path.join(output_folder, f"{base_video_name}_sequence_musiq_niqe")
                os.makedirs(sequence_dir, exist_ok=True)

                for frame_data in frame_buffer:
                    file_path = os.path.join(sequence_dir, f"frame_{frame_data['frame_num']:06d}.png")
                    cv2.imwrite(file_path, frame_data['frame'])

                break  # Sequence found, exit the loop for this video

    # --- 3. Final Status Report ---
    if len(frame_buffer) < sequence_length:
        print(f"--- FAILED to find a full sequence in {base_video_name}. Max consecutive frames found: {len(frame_buffer)} ---")

    cap.release()

if __name__ == "__main__":
    # --- 1. Configuration ---
    scenes_input_folder = r"movie_scenes\Beast-AdaptiveDetector"
    final_dataset_folder = r"movie_frames\Beast-AdaptiveDetector"

    if not os.path.isdir(scenes_input_folder):
        print(f"Error: Input directory not found at {scenes_input_folder}")
    else:
        # --- 2. Setup IQA Models ---
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Initializing IQA models on device: {device}")

        try:
            musiq_metric = pyiqa.create_metric('musiq', device=device)
            niqe_metric = pyiqa.create_metric('niqe', device=device)
        except Exception as e:
            print(f"Failed to create IQA models: {e}")
            exit()

        # --- 3. Process Videos ---
        print("Starting sequence extraction...")
        video_files = sorted([f for f in os.listdir(scenes_input_folder) if f.endswith((".mp4", ".avi", ".mov", ".mkv"))])

        for video_file in video_files:
            video_path = os.path.join(scenes_input_folder, video_file)
            find_first_sequence_musiq_niqe(
                video_path,
                final_dataset_folder,
                musiq_metric=musiq_metric,
                niqe_metric=niqe_metric,
                device=device
            )

        print("\nAll scenes processed.")
